In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import all files to notebook

In [2]:
train = pd.read_csv(r'..\data\house prediction\train.csv',index_col = 'Id')
test = pd.read_csv(r'..\data\house prediction\test.csv',index_col = 'Id')
ans = pd.read_csv(r'..\data\house prediction\sample_submission.csv',index_col = 'Id')

In [ ]:
# test = pd.merge(test,ans,on= 'Id')

In [ ]:
# train.info()

In [ ]:
null = []
for i in train.columns:
    val = pd.value_counts(train[i].isna())
    if val[0]/len(train) <= 0.6:
        null.append(i)

print("The columns must delete are : ",null)

In [ ]:
train_delete_col = train.drop(columns=null)
test_delete_col = test.drop(columns=null)

In [ ]:
# train_clean = train_delete_col.fillna(np.median)
# test_clean = test_delete_col.fillna(np.median)
train_clean = train_delete_col.fillna(train_delete_col.median())
train_clean = train_clean.dropna(axis = 0)
# test_clean = test_delete_col.fillna(train_delete_col.median(),inplace = True)

In [ ]:
train_clean.info()

In [3]:
def feature_select(data):
    null = []
    for i in train.columns:
        val = pd.value_counts(train[i].isna())
        if val[0]/len(train) <= 0.6:
            null.append(i)
    data.drop(columns=null,inplace = True)
    data.fillna(data.median(),inplace = True)
    data.dropna(axis = 0,inplace = True)
    return data

In [10]:
object_col = train_clean.select_dtypes(include='object').columns

In [12]:
object_col

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')

In [8]:
from sklearn.preprocessing import LabelEncoder
def encoder(data):
    encoder_data = []
    le = LabelEncoder()
    for i in data.columns:
        encoder_data.append(le.fit_transform(data[i]))
    return np.transpose(encoder_data)
#     return pd.DataFrame(np.reshape(np.array(encoder_data),(row,columns)),columns=data_col,index=data_index) 

In [13]:
# train_obj_encoder = encoder(train_clean[object_col])
#test_obj_encoder = encoder(test_clean[object_col])
encoder(train_clean[object_col])
# le = LabelEncoder()
# le.fit_transform(train_clean['Street'])

array([[3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 0, ..., 2, 8, 4],
       ...,
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4]])

In [ ]:
train_clean.info()

In [ ]:
train_completely_clean = pd.merge(train_clean.drop(columns = object_col),train_obj_encoder,how='inner',on = 'Id')

In [ ]:
test_obj_encoder = encoder(test_clean[object_col])
test_completely_clean = pd.merge(test_clean.drop(columns = object_col),test_obj_encoder,how='inner',on = 'Id')

In [ ]:
data_corr = train_completely_clean.corr()
key = data_corr['SalePrice'][data_corr.SalePrice>0.3].keys()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
X_train = train_completely_clean[['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']]
y_train = train_completely_clean['SalePrice']

X_test = test_completely_clean[['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']]
y_test = test_completely_clean['SalePrice']

In [ ]:
data_corr['SalePrice'][data_corr.SalePrice>0.3]

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
y_pred = lm.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error , mean_absolute_error ,r2_score
case1_mse = mean_squared_error(y_test,y_pred)
case1_mae = mean_absolute_error(y_test,y_pred)
case1_r2  = r2_score(y_test,y_pred)
print(case1_mse)
print(case1_mae)
print(case1_r2)

# Linear regression with all variable

In [ ]:
X_train_allvar = train_completely_clean.drop(columns='SalePrice')
y_train_allvar = train_completely_clean['SalePrice']

X_test_allvar = test_completely_clean.drop(columns='SalePrice')
y_test_allvar = test_completely_clean['SalePrice']

In [ ]:
from sklearn.preprocessing import StandardScaler,Normalizer
scale = StandardScaler()
X_train_allvar = scale.fit_transform(X_train_allvar)
X_test_allvar  = scale.fit_transform(X_test_allvar)

In [ ]:
lm_allvar = LinearRegression()
lm_allvar.fit(X_train_allvar,y_train_allvar)

In [ ]:
y_pred_allvar = lm_allvar.predict(X_test_allvar)

In [ ]:
case2_mse = mean_squared_error(y_test_allvar,y_pred_allvar)
case2_mae = mean_absolute_error(y_test_allvar,y_pred_allvar)
case2_r2 = r2_score(y_pred_allvar,y_test_allvar)
print(case2_mse)
print(case2_mae)
print(case2_r2)

### Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
tree.fit(X_train_allvar,y_train_allvar)
ytree_pred = tree.predict(X_test_allvar)
tree_mse = mean_squared_error(ytree_pred,y_test_allvar)
tree_mae = mean_absolute_error(ytree_pred,y_test_allvar)
tree_r2 = r2_score(ytree_pred,y_test_allvar)
print(tree_mse)
print(tree_mae)
print(tree_r2)